In [ ]:
import pathlib

import marimo as mo
import polars as pl
import altair as alt

# PPA full-text corpus changes

Comparison of two different versions of the PPA page-level full-text corpus.

Both were exported February 2025, but one was based on older data and one on a fresh rsync copy.

What are the high level differences?

In [ ]:
PPA_DATA_DIR = pathlib.Path("data/ppa/")

ppa_corpus_newer = pl.read_ndjson(
    PPA_DATA_DIR / "ppa_corpus_2025-02-19/ppa_pages.jsonl.gz"
)
# limit to HathiTrust content only, based on source id; non-hathi ids have known patterns
ppa_corpus_newer = ppa_corpus_newer.filter(
    # EEBO-TPC work id is A followed by numbers
    # Gale/ECCO work id is CW/CB followed by numbers
    # both could have an optional -p## for excerpt start page
    ~pl.col("work_id").str.contains(r"^((A\d+)|(C[WB]\d+))(-p.+)?$")
)

ppa_corpus_newer.head(10)

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"label\":\"1\",\"tags\":[\"LEFT\"],\"id\":\"aeu.ark:/13960/t0ft9gj3x.00000001\",\"order\":1,\"text\":\"CIHM \\n Microfiche \\n Series \\n ({Monographs) \\n \\n \\n (CMH \\n Collection de( \\n microfiches i \\n (monographles) \\n \\n Canadian Instituta for Historical IMicroroproduCtions / Institut Canadian da microraproductiont historiquas \\n \\n c \\n \\n\",\"work_id\":\"aeu.ark:/13960/t0ft9gj3x\"},{\"label\":\"2\",\"tags\":[\"RIGHT\"],\"id\":\"aeu.ark:/13960/t0ft9gj3x.00000002\",\"order\":2,\"text\":\"^ \\n \\n Technical and Bibliographic Notes / Notat tichniquai er JMbhographiquat \\n \\n The Institute has attempted to obtain the best original \\n copy available for filming. Features of this copy which \\n may be bibliographically unique, which may alter any \\n of the images in the reproduction, or which may \\n significantly change the usual method of filming, are \\n checked below. \\n \\n / \\n \\n li'lnstitut a microfilm^ le meilleur eiiemplaire qu'il \\n lui a kik possible de se procurer. Les details de cet \\n exemplaire qui sont peut-4tre uniques du point de \\u00abue \\n bibliographique, qui peuvent modifier une image \\n reproduite. ou qui peuvent axiger une modification \\n dans la mithode normale de f ilmage sont indiqu^ \\n ci-dessous. \\n \\n \\n \\n Q \\n D \\n \\u25a1 \\n \\n Coloured covers/ \\n Couverture de couleur \\n \\n Covers damaged/ \\n Couverture endomm \\n \\n Covers restored and/or laminated/ \\n Couverture restaurAe et/ou pellicuMe \\n \\n Cover title missing/ \\n \\n Le titre de couverture manque \\n \\n \\u25a1 Coloured pages/ , \\n Pages de couleur \\n \\n Pages damaged/ \\n Pages endommag^ \\n \\n \\u25a1 Pages restored and/or laminated/ \\n Pages restaurtes et/ou pelliculies \\n \\n Q Pages discploured. stained or foxed/ \\n Pages dteolortes, tacheties ou piquees \\n \\n r/ \\n \\n D \\n D \\n \\n a \\n \\n D \\n \\n a \\n \\n Coloured maps/ \\n \\n Caites gAographiques en couleur \\n \\n Coloured ink (i.e. other than blue or black)/ \\n Encre de couleur (i.e. autre que bleue ou noire) \\n \\n Coloured plates and/or illustrations/ \\n Planches et/ou illustrations en couleur \\n \\n Bound with other material/ \\n Relie avec d'autres dbcuments \\n \\n Tight binding may cause shadows or distortion I \\n along interior margin/ \\n \\n La reliure serrie peut cauter de I'ombre ou de la \\n distorsion le long de la marge interieure \\n \\n Blank leaves added during restoration may iippear \\n within the text. Whenever possible, these have \\n been omitted from filming/ \\n II se peut que certaiiles pages blanches aioutto \\n lors d'une restauration apparaissent dans le texte, \\n mais, lorsque cela etait possible, cesjtages n'ont \\n pas ete f ilm^s. -^ \\\\ ^ \\n \\n Ai^ditional comments:/ \\n Commentaires supplementaires: \\n \\n \\u25a1 Pages detached/ \\n Pages d^techies \\n \\n \\n \\n n \\n \\n Showthrough/ \\n Transparence \\n \\n QuaAity of print varies/ \\n Qualite inigale de I'impression \\n \\n Continuous pagination/ \\n Pagination continue \\n \\n Includes index(es)/ \\n Comprend un (des) index \\n \\n Title on header taken from: / \\n Le titre de Ten-tCte provient: \\n \\n \\u25a1 Title page of issue \\n Page de titre de la \\n \\n ssue/- \\n \\n livraison \\n \\n I I Caption of issue/ \\n \\n n \\n \\n ,Titre de depart de la livraison \\n \\n Masthead/ \\n \\n Generique (periodiques) de la livraison \\n \\n \\n This item is filmed at the reduction ratio checked below/ \\n \\n Ce document est f ilme au taux de reduction indiqui ci<lessous. \\n \\n 10X - 14X ' 18X \\n \\n ^ \\n \\n 12^ \\n \\n 16X \\n \\n 20X \\n \\n 22X \\n \\n 26 X \\n \\n 30X \\n \\n 24X \\n \\n 28X \\n \\n 32 X \\n \\n\",\"work_id\":\"aeu.ark:/13960/t0ft9gj3x\"},{\"label\":\"3\",\"tags\":[\"LEFT\"],\"id\":\"aeu.ark:/13960/t0ft9gj3x.00000003\",\"order\":3,\"text\":\"Th\\u00ab copy filmed h\\u00abr\\u00ab has h\\u2014n raproducacTthanks \\n to tha

In [ ]:
ppa_corpus_frozen = pl.read_ndjson(
    PPA_DATA_DIR / "ppa_corpus_2025-02-03_1308/ppa_pages.jsonl.gz"
)
ppa_corpus_frozen = ppa_corpus_frozen.with_columns(
    work_id_prefix=pl.col("work_id").str.slice(0, 2)
).filter(~pl.col("work_id").str.contains(r"^((A\d+)|(C[WB]\d+))(-p.+)?$"))

ppa_corpus_frozen.head()

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"label\":\"1\",\"tags\":[\"LEFT\"],\"id\":\"aeu.ark:/13960/t0ft9gj3x.00000001\",\"order\":1,\"text\":\"CIHM \\n Microfiche \\n Series \\n ({Monographs) \\n \\n \\n (CMH \\n Collection de( \\n microfiches i \\n (monographles) \\n \\n Canadian Instituta for Historical IMicroroproduCtions / Institut Canadian da microraproductiont historiquas \\n \\n c \\n \\n\",\"work_id\":\"aeu.ark:/13960/t0ft9gj3x\",\"work_id_prefix\":\"ae\"},{\"label\":\"2\",\"tags\":[\"RIGHT\"],\"id\":\"aeu.ark:/13960/t0ft9gj3x.00000002\",\"order\":2,\"text\":\"^ \\n \\n Technical and Bibliographic Notes / Notat tichniquai er JMbhographiquat \\n \\n The Institute has attempted to obtain the best original \\n copy available for filming. Features of this copy which \\n may be bibliographically unique, which may alter any \\n of the images in the reproduction, or which may \\n significantly change the usual method of filming, are \\n checked below. \\n \\n / \\n \\n li'lnstitut a microfilm^ le meilleur eiiemplaire qu'il \\n lui a kik possible de se procurer. Les details de cet \\n exemplaire qui sont peut-4tre uniques du point de \\u00abue \\n bibliographique, qui peuvent modifier une image \\n reproduite. ou qui peuvent axiger une modification \\n dans la mithode normale de f ilmage sont indiqu^ \\n ci-dessous. \\n \\n \\n \\n Q \\n D \\n \\u25a1 \\n \\n Coloured covers/ \\n Couverture de couleur \\n \\n Covers damaged/ \\n Couverture endomm \\n \\n Covers restored and/or laminated/ \\n Couverture restaurAe et/ou pellicuMe \\n \\n Cover title missing/ \\n \\n Le titre de couverture manque \\n \\n \\u25a1 Coloured pages/ , \\n Pages de couleur \\n \\n Pages damaged/ \\n Pages endommag^ \\n \\n \\u25a1 Pages restored and/or laminated/ \\n Pages restaurtes et/ou pelliculies \\n \\n Q Pages discploured. stained or foxed/ \\n Pages dteolortes, tacheties ou piquees \\n \\n r/ \\n \\n D \\n D \\n \\n a \\n \\n D \\n \\n a \\n \\n Coloured maps/ \\n \\n Caites gAographiques en couleur \\n \\n Coloured ink (i.e. other than blue or black)/ \\n Encre de couleur (i.e. autre que bleue ou noire) \\n \\n Coloured plates and/or illustrations/ \\n Planches et/ou illustrations en couleur \\n \\n Bound with other material/ \\n Relie avec d'autres dbcuments \\n \\n Tight binding may cause shadows or distortion I \\n along interior margin/ \\n \\n La reliure serrie peut cauter de I'ombre ou de la \\n distorsion le long de la marge interieure \\n \\n Blank leaves added during restoration may iippear \\n within the text. Whenever possible, these have \\n been omitted from filming/ \\n II se peut que certaiiles pages blanches aioutto \\n lors d'une restauration apparaissent dans le texte, \\n mais, lorsque cela etait possible, cesjtages n'ont \\n pas ete f ilm^s. -^ \\\\ ^ \\n \\n Ai^ditional comments:/ \\n Commentaires supplementaires: \\n \\n \\u25a1 Pages detached/ \\n Pages d^techies \\n \\n \\n \\n n \\n \\n Showthrough/ \\n Transparence \\n \\n QuaAity of print varies/ \\n Qualite inigale de I'impression \\n \\n Continuous pagination/ \\n Pagination continue \\n \\n Includes index(es)/ \\n Comprend un (des) index \\n \\n Title on header taken from: / \\n Le titre de Ten-tCte provient: \\n \\n \\u25a1 Title page of issue \\n Page de titre de la \\n \\n ssue/- \\n \\n livraison \\n \\n I I Caption of issue/ \\n \\n n \\n \\n ,Titre de depart de la livraison \\n \\n Masthead/ \\n \\n Generique (periodiques) de la livraison \\n \\n \\n This item is filmed at the reduction ratio checked below/ \\n \\n Ce document est f ilme au taux de reduction indiqui ci<lessous. \\n \\n 10X - 14X ' 18X \\n \\n ^ \\n \\n 12^ \\n \\n 16X \\n \\n 20X \\n \\n 22X \\n \\n 26 X \\n \\n 30X \\n \\n 24X \\n \\n 28X \\n \\n 32 X \\n \\n\",\"work_id\":\"aeu.ark:/13960/t0ft9gj3x\",\"work_id_prefix\":\"ae\"},{\"label\":\"3\",\"tags\":[\"LEFT\"],\"id\":\"aeu.ark:/13960/t0ft9gj3x.00000003\",\"order\":3,\"text\":\"Th\\u00ab copy filmed h\\u00a

How many pages in these two versions? Any difference by raw page count?

In [ ]:
total_ht_pages_frozen = ppa_corpus_frozen.height
total_ht_pages_newer = ppa_corpus_newer.height

mo.md(f"""
- {total_ht_pages_frozen:,} pages in frozen production data
- {total_ht_pages_newer:,} pages in 2025-02-19 rsync copy

{total_ht_pages_frozen - total_ht_pages_newer:,} more pages in the earlier version

""")

1,569,407 pages in frozen production data 
 1,524,217 pages in 2025-02-19 rsync copy 
 
 45,190 more pages in the earlier version

Are there changes in the number of volumes ?

In [ ]:
total_works_frozen = ppa_corpus_frozen["work_id"].unique().len()
total_works_newer = ppa_corpus_newer["work_id"].unique().len()

mo.md(f"""
- {total_works_frozen:,} total unique HT ids in frozen production data
- {total_works_newer:,} total unique HT ids in rsync copy

difference: {total_works_frozen - total_works_newer} volumes
""")
# print(total_works_202404, total_works_202502)

5,538 total unique HT ids in frozen production data 
 5,320 total unique HT ids in rsync copy 
 
 difference: 218 volumes

First compare by page ids. How many match, and of those how many have the same text content?

In [ ]:
pageid_join = ppa_corpus_frozen.join(
    ppa_corpus_newer, on="id", how="inner"
).with_columns(text_equal=pl.col("text").eq(pl.col("text_right")))

pageid_join.head(10)

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"label\":\"1\",\"tags\":[\"LEFT\"],\"id\":\"aeu.ark:/13960/t0ft9gj3x.00000001\",\"order\":1,\"text\":\"CIHM \\n Microfiche \\n Series \\n ({Monographs) \\n \\n \\n (CMH \\n Collection de( \\n microfiches i \\n (monographles) \\n \\n Canadian Instituta for Historical IMicroroproduCtions / Institut Canadian da microraproductiont historiquas \\n \\n c \\n \\n\",\"work_id\":\"aeu.ark:/13960/t0ft9gj3x\",\"work_id_prefix\":\"ae\",\"label_right\":\"1\",\"tags_right\":[\"LEFT\"],\"order_right\":1,\"text_right\":\"CIHM \\n Microfiche \\n Series \\n ({Monographs) \\n \\n \\n (CMH \\n Collection de( \\n microfiches i \\n (monographles) \\n \\n Canadian Instituta for Historical IMicroroproduCtions / Institut Canadian da microraproductiont historiquas \\n \\n c \\n \\n\",\"work_id_right\":\"aeu.ark:/13960/t0ft9gj3x\",\"text_equal\":true},{\"label\":\"2\",\"tags\":[\"RIGHT\"],\"id\":\"aeu.ark:/13960/t0ft9gj3x.00000002\",\"order\":2,\"text\":\"^ \\n \\n Technical and Bibliographic Notes / Notat tichniquai er JMbhographiquat \\n \\n The Institute has attempted to obtain the best original \\n copy available for filming. Features of this copy which \\n may be bibliographically unique, which may alter any \\n of the images in the reproduction, or which may \\n significantly change the usual method of filming, are \\n checked below. \\n \\n / \\n \\n li'lnstitut a microfilm^ le meilleur eiiemplaire qu'il \\n lui a kik possible de se procurer. Les details de cet \\n exemplaire qui sont peut-4tre uniques du point de \\u00abue \\n bibliographique, qui peuvent modifier une image \\n reproduite. ou qui peuvent axiger une modification \\n dans la mithode normale de f ilmage sont indiqu^ \\n ci-dessous. \\n \\n \\n \\n Q \\n D \\n \\u25a1 \\n \\n Coloured covers/ \\n Couverture de couleur \\n \\n Covers damaged/ \\n Couverture endomm \\n \\n Covers restored and/or laminated/ \\n Couverture restaurAe et/ou pellicuMe \\n \\n Cover title missing/ \\n \\n Le titre de couverture manque \\n \\n \\u25a1 Coloured pages/ , \\n Pages de couleur \\n \\n Pages damaged/ \\n Pages endommag^ \\n \\n \\u25a1 Pages restored and/or laminated/ \\n Pages restaurtes et/ou pelliculies \\n \\n Q Pages discploured. stained or foxed/ \\n Pages dteolortes, tacheties ou piquees \\n \\n r/ \\n \\n D \\n D \\n \\n a \\n \\n D \\n \\n a \\n \\n Coloured maps/ \\n \\n Caites gAographiques en couleur \\n \\n Coloured ink (i.e. other than blue or black)/ \\n Encre de couleur (i.e. autre que bleue ou noire) \\n \\n Coloured plates and/or illustrations/ \\n Planches et/ou illustrations en couleur \\n \\n Bound with other material/ \\n Relie avec d'autres dbcuments \\n \\n Tight binding may cause shadows or distortion I \\n along interior margin/ \\n \\n La reliure serrie peut cauter de I'ombre ou de la \\n distorsion le long de la marge interieure \\n \\n Blank leaves added during restoration may iippear \\n within the text. Whenever possible, these have \\n been omitted from filming/ \\n II se peut que certaiiles pages blanches aioutto \\n lors d'une restauration apparaissent dans le texte, \\n mais, lorsque cela etait possible, cesjtages n'ont \\n pas ete f ilm^s. -^ \\\\ ^ \\n \\n Ai^ditional comments:/ \\n Commentaires supplementaires: \\n \\n \\u25a1 Pages detached/ \\n Pages d^techies \\n \\n \\n \\n n \\n \\n Showthrough/ \\n Transparence \\n \\n QuaAity of print varies/ \\n Qualite inigale de I'impression \\n \\n Continuous pagination/ \\n Pagination continue \\n \\n Includes index(es)/ \\n Comprend un (des) index \\n \\n Title on header taken from: / \\n Le titre de Ten-tCte provient: \\n \\n \\u25a1 Title page of issue \\n Page de titre de la \\n \\n ssue/- \\n \\n livraison \\n \\n I I Caption of issue/ \\n \\n n \\n \\n ,Titre de depart de la livraison \\n \\n Masthead/ \\n \\n Generique (periodiques) de la livraison \\n \\n \\n This item is filmed at the reduction ratio checked below/ \\n \\n Ce document 

In [ ]:
total_shared_pageids = pageid_join.height
total_pages_text_eq = pageid_join.filter(pl.col("text_equal")).height

mo.md(f"""{total_shared_pageids:,} page ids in both versions

{total_pages_text_eq:,}  pages with unchanged text content — {(total_pages_text_eq / total_shared_pageids) * 100:.1f}%!

for {total_shared_pageids - total_pages_text_eq:,} pages ({((total_shared_pageids - total_pages_text_eq) / total_shared_pageids) * 100:.1f}%) the text has changed
""")

1,523,781 page ids in both versions 
 849,615 pages with unchanged text content — 55.8%! 
 for 674,166 pages (44.2%) the text has changed

In [ ]:
# how many pages are null? 38,505
# ppa_corpus_202404.filter(pl.col("text").is_null())
# how many are empty string? 12,469
ppa_corpus_newer.filter(pl.col("text").str.strip_chars().eq(""))

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"label\":\"5\",\"tags\":[\"LEFT\"],\"id\":\"aeu.ark:/13960/t3514pf3k.00000005\",\"order\":5,\"text\":\"\\n\",\"work_id\":\"aeu.ark:/13960/t3514pf3k\"},{\"label\":\"293\",\"tags\":[\"LEFT\"],\"id\":\"aeu.ark:/13960/t8v98vx29.00000293\",\"order\":293,\"text\":\"\\n\",\"work_id\":\"aeu.ark:/13960/t8v98vx29\"},{\"label\":\"2\",\"tags\":[\"LEFT\"],\"id\":\"bc.ark:/13960/t2p63fq11.00000002\",\"order\":2,\"text\":\"\\n \\n \\n\",\"work_id\":\"bc.ark:/13960/t2p63fq11\"},{\"label\":\"4\",\"tags\":[\"LEFT\"],\"id\":\"bc.ark:/13960/t2p63fq11.00000004\",\"order\":4,\"text\":\"\\n\",\"work_id\":\"bc.ark:/13960/t2p63fq11\"},{\"label\":\"1\",\"tags\":[\"FRONT_COVER\",\"IMAGE_ON_PAGE\",\"IMPLICIT_PAGE_NUMBER\"],\"id\":\"chi.084960397.00000001\",\"order\":1,\"text\":\"\\n\",\"work_id\":\"chi.084960397\"},{\"label\":\"6\",\"tags\":[\"IMAGE_ON_PAGE\",\"IMPLICIT_PAGE_NUMBER\"],\"id\":\"chi.084960834.00000006\",\"order\":6,\"text\":\"\\n\",\"work_id\":\"chi.084960834\"},{\"label\":\"71\",\"tags\":[\"IMAGE_ON_PAGE\",\"IMPLICIT_PAGE_NUMBER\"],\"id\":\"chi.084960834.00000071\",\"order\":71,\"text\":\"\\n\",\"work_id\":\"chi.084960834\"},{\"label\":\"72\",\"tags\":[\"IMAGE_ON_PAGE\",\"IMPLICIT_PAGE_NUMBER\"],\"id\":\"chi.084960834.00000072\",\"order\":72,\"text\":\"\\n\",\"work_id\":\"chi.084960834\"},{\"label\":\"1\",\"tags\":[\"FRONT_COVER\",\"IMAGE_ON_PAGE\",\"IMPLICIT_PAGE_NUMBER\"],\"id\":\"chi.085136836.00000001\",\"order\":1,\"text\":\"\\n\",\"work_id\":\"chi.085136836\"},{\"label\":\"3\",\"tags\":[\"IMAGE_ON_PAGE\",\"IMPLICIT_PAGE_NUMBER\"],\"id\":\"chi.085137191.00000003\",\"order\":3,\"text\":\"\\n\",\"work_id\":\"chi.085137191\"}]"' data-total-rows='9031' data-total-columns='6' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["label", ["string", "str"]], ["tags", ["unknown", "list[str]"]], ["id", ["string", "str"]], ["order", ["integer", "i64"]], ["text", ["string", "str"]], ["work_id", ["string", "str"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='true' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

In [ ]:
# what if we join on exact text?
# DUH filter out empty pages so they don't all match each other
# join on matching page and same work id
pagetext_join = (
    ppa_corpus_frozen.filter(~pl.col("text").is_null())
    .filter(~pl.col("text").str.strip_chars().eq(""))
    .join(
        ppa_corpus_newer.filter(~pl.col("text").is_null()).filter(
            ~pl.col("text").str.strip_chars().eq("")
        ),
        on=["text", "work_id"],
        how="inner",
    )
    .with_columns(id_equal=pl.col("id").eq(pl.col("id_right")))
)

pagetext_join.head(10)

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"label\":\"1\",\"tags\":[\"LEFT\"],\"id\":\"aeu.ark:/13960/t0ft9gj3x.00000001\",\"order\":1,\"text\":\"CIHM \\n Microfiche \\n Series \\n ({Monographs) \\n \\n \\n (CMH \\n Collection de( \\n microfiches i \\n (monographles) \\n \\n Canadian Instituta for Historical IMicroroproduCtions / Institut Canadian da microraproductiont historiquas \\n \\n c \\n \\n\",\"work_id\":\"aeu.ark:/13960/t0ft9gj3x\",\"work_id_prefix\":\"ae\",\"label_right\":\"1\",\"tags_right\":[\"LEFT\"],\"id_right\":\"aeu.ark:/13960/t0ft9gj3x.00000001\",\"order_right\":1,\"id_equal\":true},{\"label\":\"2\",\"tags\":[\"RIGHT\"],\"id\":\"aeu.ark:/13960/t0ft9gj3x.00000002\",\"order\":2,\"text\":\"^ \\n \\n Technical and Bibliographic Notes / Notat tichniquai er JMbhographiquat \\n \\n The Institute has attempted to obtain the best original \\n copy available for filming. Features of this copy which \\n may be bibliographically unique, which may alter any \\n of the images in the reproduction, or which may \\n significantly change the usual method of filming, are \\n checked below. \\n \\n / \\n \\n li'lnstitut a microfilm^ le meilleur eiiemplaire qu'il \\n lui a kik possible de se procurer. Les details de cet \\n exemplaire qui sont peut-4tre uniques du point de \\u00abue \\n bibliographique, qui peuvent modifier une image \\n reproduite. ou qui peuvent axiger une modification \\n dans la mithode normale de f ilmage sont indiqu^ \\n ci-dessous. \\n \\n \\n \\n Q \\n D \\n \\u25a1 \\n \\n Coloured covers/ \\n Couverture de couleur \\n \\n Covers damaged/ \\n Couverture endomm \\n \\n Covers restored and/or laminated/ \\n Couverture restaurAe et/ou pellicuMe \\n \\n Cover title missing/ \\n \\n Le titre de couverture manque \\n \\n \\u25a1 Coloured pages/ , \\n Pages de couleur \\n \\n Pages damaged/ \\n Pages endommag^ \\n \\n \\u25a1 Pages restored and/or laminated/ \\n Pages restaurtes et/ou pelliculies \\n \\n Q Pages discploured. stained or foxed/ \\n Pages dteolortes, tacheties ou piquees \\n \\n r/ \\n \\n D \\n D \\n \\n a \\n \\n D \\n \\n a \\n \\n Coloured maps/ \\n \\n Caites gAographiques en couleur \\n \\n Coloured ink (i.e. other than blue or black)/ \\n Encre de couleur (i.e. autre que bleue ou noire) \\n \\n Coloured plates and/or illustrations/ \\n Planches et/ou illustrations en couleur \\n \\n Bound with other material/ \\n Relie avec d'autres dbcuments \\n \\n Tight binding may cause shadows or distortion I \\n along interior margin/ \\n \\n La reliure serrie peut cauter de I'ombre ou de la \\n distorsion le long de la marge interieure \\n \\n Blank leaves added during restoration may iippear \\n within the text. Whenever possible, these have \\n been omitted from filming/ \\n II se peut que certaiiles pages blanches aioutto \\n lors d'une restauration apparaissent dans le texte, \\n mais, lorsque cela etait possible, cesjtages n'ont \\n pas ete f ilm^s. -^ \\\\ ^ \\n \\n Ai^ditional comments:/ \\n Commentaires supplementaires: \\n \\n \\u25a1 Pages detached/ \\n Pages d^techies \\n \\n \\n \\n n \\n \\n Showthrough/ \\n Transparence \\n \\n QuaAity of print varies/ \\n Qualite inigale de I'impression \\n \\n Continuous pagination/ \\n Pagination continue \\n \\n Includes index(es)/ \\n Comprend un (des) index \\n \\n Title on header taken from: / \\n Le titre de Ten-tCte provient: \\n \\n \\u25a1 Title page of issue \\n Page de titre de la \\n \\n ssue/- \\n \\n livraison \\n \\n I I Caption of issue/ \\n \\n n \\n \\n ,Titre de depart de la livraison \\n \\n Masthead/ \\n \\n Generique (periodiques) de la livraison \\n \\n \\n This item is filmed at the reduction ratio checked below/ \\n \\n Ce document est f ilme au taux de reduction indiqui ci<lessous. \\n \\n 10X - 14X ' 18X \\n \\n ^ \\n \\n 12^ \\n \\n 16X \\n \\n 20X \\n \\n 22X \\n \\n 26 X \\n \\n 30X \\n \\n 24X \\n \\n 28X \\n \\n 32 X \\n \\n\",\"work_id\":\"aeu.ark:/13960/t0ft9gj3x\",\"work_id_prefix\":\"ae\",

In [ ]:
total_shared_pagetext = pagetext_join.height
total_pagetext_id_eq = pagetext_join.filter(pl.col("id_equal")).height

mo.md(f"""{total_shared_pagetext:,} pages with exactly matching text in both versions

{total_pagetext_id_eq:,}  pages with matching text have unchanged page ids — {(total_pagetext_id_eq / total_shared_pagetext) * 100:.1f}% of this set; {(total_pagetext_id_eq / total_ht_pages_frozen) * 100:.1f}% of frozen corpus

for {total_shared_pagetext - total_pagetext_id_eq:,} pages ({((total_shared_pagetext - total_pagetext_id_eq) / total_shared_pagetext) * 100:.1f}%) the id has changed
""")

855,930 pages with exactly matching text in both versions 
 841,507 pages with matching text have unchanged page ids — 98.3% of this set; 53.6% of frozen corpus 
 for 14,423 pages (1.7%) the id has changed

### When was PPA data from HathiTrust last modified

In [ ]:
from pairtree import path2id


def path_to_htid(filepath):
    filepath = pathlib.Path(filepath)
    prefix = filepath.parts[0]
    return f"{prefix}.{path2id(filepath.stem.rsplit('.')[0])}"


def path_suffixes(filepath):
    filepath = pathlib.Path(filepath)
    return "".join(filepath.suffixes)


lastmod_df = pl.read_csv(
    "data/ppa/ppa_htfiles_lastmod.csv",
    schema_overrides={"last_modified": pl.datatypes.Datetime},
).with_columns(
    last_mod_day=pl.col("last_modified").cast(pl.datatypes.Date),
    htid=pl.col("filename").map_elements(
        path_to_htid, return_dtype=pl.datatypes.String
    ),
    file_type=pl.col("filename").map_elements(
        path_suffixes, return_dtype=pl.datatypes.String
    ),
)


lastmod_mets_df = lastmod_df.filter(pl.col("file_type").eq(".mets.xml"))
lastmod_text_df = lastmod_df.filter(pl.col("file_type").eq(".zip"))


lastmod_df.head(10)

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"filename\":\"inu/pairtree_root/39/00/00/02/41/94/84/39000002419484/39000002419484.mets.xml\",\"mtime\":1677112122.291734,\"last_modified\":\"2023-02-22 19:28:42.291734\",\"last_mod_day\":\"2023-02-22\",\"htid\":\"inu.39000002419484\",\"file_type\":\".mets.xml\"},{\"filename\":\"inu/pairtree_root/39/00/00/02/41/94/84/39000002419484/39000002419484.zip\",\"mtime\":1677325441.798539,\"last_modified\":\"2023-02-25 06:44:01.798539\",\"last_mod_day\":\"2023-02-25\",\"htid\":\"inu.39000002419484\",\"file_type\":\".zip\"},{\"filename\":\"inu/pairtree_root/39/00/00/00/14/53/47/39000000145347/39000000145347.zip\",\"mtime\":1692529532.654552,\"last_modified\":\"2023-08-20 07:05:32.654552\",\"last_mod_day\":\"2023-08-20\",\"htid\":\"inu.39000000145347\",\"file_type\":\".zip\"},{\"filename\":\"inu/pairtree_root/39/00/00/00/14/53/47/39000000145347/39000000145347.mets.xml\",\"mtime\":1692292242.639291,\"last_modified\":\"2023-08-17 13:10:42.639291\",\"last_mod_day\":\"2023-08-17\",\"htid\":\"inu.39000000145347\",\"file_type\":\".mets.xml\"},{\"filename\":\"inu/pairtree_root/39/00/00/03/78/45/06/39000003784506/39000003784506.zip\",\"mtime\":1678190013.552734,\"last_modified\":\"2023-03-07 06:53:33.552734\",\"last_mod_day\":\"2023-03-07\",\"htid\":\"inu.39000003784506\",\"file_type\":\".zip\"},{\"filename\":\"inu/pairtree_root/39/00/00/03/78/45/06/39000003784506/39000003784506.mets.xml\",\"mtime\":1677947786.937365,\"last_modified\":\"2023-03-04 11:36:26.937365\",\"last_mod_day\":\"2023-03-04\",\"htid\":\"inu.39000003784506\",\"file_type\":\".mets.xml\"},{\"filename\":\"inu/pairtree_root/39/00/00/01/47/01/73/39000001470173/39000001470173.zip\",\"mtime\":1692875086.694633,\"last_modified\":\"2023-08-24 07:04:46.694633\",\"last_mod_day\":\"2023-08-24\",\"htid\":\"inu.39000001470173\",\"file_type\":\".zip\"},{\"filename\":\"inu/pairtree_root/39/00/00/01/47/01/73/39000001470173/39000001470173.mets.xml\",\"mtime\":1692613640.924281,\"last_modified\":\"2023-08-21 06:27:20.924281\",\"last_mod_day\":\"2023-08-21\",\"htid\":\"inu.39000001470173\",\"file_type\":\".mets.xml\"},{\"filename\":\"inu/pairtree_root/39/00/00/04/51/67/66/39000004516766/39000004516766.zip\",\"mtime\":1671967401.201495,\"last_modified\":\"2022-12-25 06:23:21.201495\",\"last_mod_day\":\"2022-12-25\",\"htid\":\"inu.39000004516766\",\"file_type\":\".zip\"},{\"filename\":\"inu/pairtree_root/39/00/00/04/51/67/66/39000004516766/39000004516766.mets.xml\",\"mtime\":1671691227.56921,\"last_modified\":\"2022-12-22 01:40:27.569210\",\"last_mod_day\":\"2022-12-22\",\"htid\":\"inu.39000004516766\",\"file_type\":\".mets.xml\"}]"' data-total-rows='10' data-total-columns='6' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["filename", ["string", "str"]], ["mtime", ["number", "f64"]], ["last_modified", ["datetime", "datetime[\u03bcs]"]], ["last_mod_day", ["date", "date"]], ["htid", ["string", "str"]], ["file_type", ["string", "str"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='false' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

In [ ]:
mo.md(f"""{lastmod_df.height:,} total rows; {lastmod_mets_df.height:,} METS xml files, {lastmod_text_df.height:,} ZIP files (text)""")

10,734 total rows; 5,367 METS xml files, 5,367 ZIP files (text)

In [ ]:
# filter to just the METS since that corresponds to volume update time
ppa_lastmod_chart = (
    alt.Chart(lastmod_mets_df)
    .mark_bar(width=10, color="#57c4c4")
    .encode(
        x=alt.X("last_mod_day:T", title="Date last modified").axis(
            tickCount="year"
        ),
        y=alt.Y("count()", title="Number of records"),
    )
    .properties(height=100, width=840)
)


ppa_lastmod_chart.save("figures/ppa_hathitrust_lastmodified.pdf")
ppa_lastmod_chart

How many volumes last modified in 2024?

In [ ]:
lastmod_mets_df.filter(pl.col("last_mod_day").gt(pl.datetime(2024, 1, 1)))

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"filename\":\"inu/pairtree_root/32/00/00/00/68/31/38/32000000683138/32000000683138.mets.xml\",\"mtime\":1705655375.958763,\"last_modified\":\"2024-01-19 04:09:35.958763\",\"last_mod_day\":\"2024-01-19\",\"htid\":\"inu.32000000683138\",\"file_type\":\".mets.xml\"},{\"filename\":\"inu/pairtree_root/30/00/01/25/50/71/15/30000125507115/30000125507115.mets.xml\",\"mtime\":1705185218.476019,\"last_modified\":\"2024-01-13 17:33:38.476019\",\"last_mod_day\":\"2024-01-13\",\"htid\":\"inu.30000125507115\",\"file_type\":\".mets.xml\"},{\"filename\":\"inu/pairtree_root/30/00/01/20/72/76/84/30000120727684/30000120727684.mets.xml\",\"mtime\":1705400779.649384,\"last_modified\":\"2024-01-16 05:26:19.649384\",\"last_mod_day\":\"2024-01-16\",\"htid\":\"inu.30000120727684\",\"file_type\":\".mets.xml\"},{\"filename\":\"inu/pairtree_root/30/00/01/20/49/71/89/30000120497189/30000120497189.mets.xml\",\"mtime\":1708520185.186693,\"last_modified\":\"2024-02-21 07:56:25.186693\",\"last_mod_day\":\"2024-02-21\",\"htid\":\"inu.30000120497189\",\"file_type\":\".mets.xml\"},{\"filename\":\"inu/pairtree_root/30/00/01/08/33/45/03/30000108334503/30000108334503.mets.xml\",\"mtime\":1704996603.343203,\"last_modified\":\"2024-01-11 13:10:03.343203\",\"last_mod_day\":\"2024-01-11\",\"htid\":\"inu.30000108334503\",\"file_type\":\".mets.xml\"},{\"filename\":\"inu/pairtree_root/30/00/01/12/84/11/13/30000112841113/30000112841113.mets.xml\",\"mtime\":1704320510.645725,\"last_modified\":\"2024-01-03 17:21:50.645725\",\"last_mod_day\":\"2024-01-03\",\"htid\":\"inu.30000112841113\",\"file_type\":\".mets.xml\"},{\"filename\":\"uc1/pairtree_root/b3/11/02/72/b3110272/b3110272.mets.xml\",\"mtime\":1713590679.62718,\"last_modified\":\"2024-04-20 01:24:39.627180\",\"last_mod_day\":\"2024-04-20\",\"htid\":\"uc1.b3110272\",\"file_type\":\".mets.xml\"},{\"filename\":\"uc1/pairtree_root/b3/11/09/27/b3110927/b3110927.mets.xml\",\"mtime\":1714328198.510207,\"last_modified\":\"2024-04-28 14:16:38.510207\",\"last_mod_day\":\"2024-04-28\",\"htid\":\"uc1.b3110927\",\"file_type\":\".mets.xml\"},{\"filename\":\"uc1/pairtree_root/b3/14/61/04/b3146104/b3146104.mets.xml\",\"mtime\":1713607064.359596,\"last_modified\":\"2024-04-20 05:57:44.359596\",\"last_mod_day\":\"2024-04-20\",\"htid\":\"uc1.b3146104\",\"file_type\":\".mets.xml\"},{\"filename\":\"uc1/pairtree_root/b3/13/68/96/b3136896/b3136896.mets.xml\",\"mtime\":1713599397.304782,\"last_modified\":\"2024-04-20 03:49:57.304782\",\"last_mod_day\":\"2024-04-20\",\"htid\":\"uc1.b3136896\",\"file_type\":\".mets.xml\"}]"' data-total-rows='411' data-total-columns='6' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["filename", ["string", "str"]], ["mtime", ["number", "f64"]], ["last_modified", ["datetime", "datetime[\u03bcs]"]], ["last_mod_day", ["date", "date"]], ["htid", ["string", "str"]], ["file_type", ["string", "str"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='true' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

What is the most recent last modified time in this set of files?

In [ ]:
lastmod_mets_df.sort("last_modified", descending=True)

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"filename\":\"mdp/pairtree_root/39/01/50/30/83/54/28/39015030835428/39015030835428.mets.xml\",\"mtime\":1731489854.314039,\"last_modified\":\"2024-11-13 04:24:14.314039\",\"last_mod_day\":\"2024-11-13\",\"htid\":\"mdp.39015030835428\",\"file_type\":\".mets.xml\"},{\"filename\":\"mdp/pairtree_root/39/01/50/11/82/71/39/39015011827139/39015011827139.mets.xml\",\"mtime\":1726746975.35789,\"last_modified\":\"2024-09-19 07:56:15.357890\",\"last_mod_day\":\"2024-09-19\",\"htid\":\"mdp.39015011827139\",\"file_type\":\".mets.xml\"},{\"filename\":\"mdp/pairtree_root/39/01/50/05/63/81/12/39015005638112/39015005638112.mets.xml\",\"mtime\":1726745164.041096,\"last_modified\":\"2024-09-19 07:26:04.041096\",\"last_mod_day\":\"2024-09-19\",\"htid\":\"mdp.39015005638112\",\"file_type\":\".mets.xml\"},{\"filename\":\"mdp/pairtree_root/39/01/50/02/72/50/45/39015002725045/39015002725045.mets.xml\",\"mtime\":1724819138.047614,\"last_modified\":\"2024-08-28 00:25:38.047614\",\"last_mod_day\":\"2024-08-28\",\"htid\":\"mdp.39015002725045\",\"file_type\":\".mets.xml\"},{\"filename\":\"uc1/pairtree_root/$b/15/67/00/$b156700/$b156700.mets.xml\",\"mtime\":1723176634.526531,\"last_modified\":\"2024-08-09 00:10:34.526531\",\"last_mod_day\":\"2024-08-09\",\"htid\":\"uc1.$b156700\",\"file_type\":\".mets.xml\"},{\"filename\":\"uc1/pairtree_root/$b/31/65/7/$b31657/$b31657.mets.xml\",\"mtime\":1722336275.730529,\"last_modified\":\"2024-07-30 06:44:35.730529\",\"last_mod_day\":\"2024-07-30\",\"htid\":\"uc1.$b31657\",\"file_type\":\".mets.xml\"},{\"filename\":\"uc1/pairtree_root/32/10/60/01/55/93/81/32106001559381/32106001559381.mets.xml\",\"mtime\":1715664468.216882,\"last_modified\":\"2024-05-14 01:27:48.216882\",\"last_mod_day\":\"2024-05-14\",\"htid\":\"uc1.32106001559381\",\"file_type\":\".mets.xml\"},{\"filename\":\"mdp/pairtree_root/39/01/50/37/31/61/82/39015037316182/39015037316182.mets.xml\",\"mtime\":1714834524.656922,\"last_modified\":\"2024-05-04 10:55:24.656922\",\"last_mod_day\":\"2024-05-04\",\"htid\":\"mdp.39015037316182\",\"file_type\":\".mets.xml\"},{\"filename\":\"uc1/pairtree_root/32/10/60/20/06/05/44/32106020060544/32106020060544.mets.xml\",\"mtime\":1714814572.689144,\"last_modified\":\"2024-05-04 05:22:52.689144\",\"last_mod_day\":\"2024-05-04\",\"htid\":\"uc1.32106020060544\",\"file_type\":\".mets.xml\"},{\"filename\":\"uc1/pairtree_root/32/10/60/20/06/05/36/32106020060536/32106020060536.mets.xml\",\"mtime\":1714814176.710264,\"last_modified\":\"2024-05-04 05:16:16.710264\",\"last_mod_day\":\"2024-05-04\",\"htid\":\"uc1.32106020060536\",\"file_type\":\".mets.xml\"}]"' data-total-rows='5367' data-total-columns='6' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["filename", ["string", "str"]], ["mtime", ["number", "f64"]], ["last_modified", ["datetime", "datetime[\u03bcs]"]], ["last_mod_day", ["date", "date"]], ["htid", ["string", "str"]], ["file_type", ["string", "str"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='true' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

In [ ]:
most_recent = lastmod_mets_df.sort("last_modified", descending=True).row(
    0, named=True
)
mo.md(f"Most recently modified file: {most_recent['last_modified']}")

Most recently modified file: 2024-11-13 04:24:14.314039